In [1]:
import gymnasium as gym


#定义环境
class MyWrapper(gym.Wrapper):

    def __init__(self):
        env = gym.make('CartPole-v1')
        super().__init__(env)
        self.env = env

    def reset(self, seed=None, options=None):
        state, info = self.env.reset()
        return state, info

    def step(self, action):
        state, reward, done, truncated, info = self.env.step(action)
        return state, reward, done, truncated, info


env = MyWrapper()

env.reset()

(array([-3.5905570e-02, -2.5838368e-02,  6.9978703e-03, -1.3193355e-05],
       dtype=float32),
 {})

In [2]:
import torch

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy


#自定义策略网络
class CustomNetwork(torch.nn.Module):

    def __init__(self,
                 feature_dim,
                 last_layer_dim_pi=64,
                 last_layer_dim_vf=64):

        super().__init__()

        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

        self.policy_net = torch.nn.Sequential(
            torch.nn.Linear(feature_dim, last_layer_dim_pi),
            torch.nn.ReLU(),
        )

        self.value_net = torch.nn.Sequential(
            torch.nn.Linear(feature_dim, last_layer_dim_vf),
            torch.nn.ReLU(),
        )

    def forward(self, features):
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features):
        return self.policy_net(features)

    def forward_critic(self, features):
        return self.value_net(features)


#使用自定义策略网络
class CustomActorCriticPolicy(ActorCriticPolicy):

    def __init__(self, observation_space, action_space, lr_schedule,
                 custom_param, *args, **kwargs):
        super().__init__(observation_space, action_space, lr_schedule, *args,
                         **kwargs)
        print('custom_param=', custom_param)
        self.ortho_init = False

    def _build_mlp_extractor(self) -> None:
        self.mlp_extractor = CustomNetwork(self.features_dim)


model = PPO(CustomActorCriticPolicy,
            env,
            policy_kwargs={'custom_param': 'lee'},
            verbose=0,
            device='cpu')

model

custom_param= lee


In [3]:
from stable_baselines3.common.evaluation import evaluate_policy

#测试
evaluate_policy(model, env, n_eval_episodes=10, deterministic=False)

C:\Users\11602\miniconda3\envs\RL\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(np.float64(18.8), np.float64(7.652450587883597))

In [4]:
#训练
model.learn(total_timesteps=2_0000, progress_bar=True)

model.save('models/自定义策略网络层')

Output()

In [5]:
model = PPO.load('models/自定义策略网络层')

evaluate_policy(model, env, n_eval_episodes=10, deterministic=False)

custom_param= lee


(np.float64(216.2), np.float64(72.24652240765641))